In [ ]:
# Check multithreading config:
Base.Threads.nthreads()

In [ ]:
# Check active package versions:
using Pkg; pkg"status"

<h1 style="text-align: center;">
    <span style="display: block; text-align: center;">
        Introduction to
    </span>
    <span style="display: block; text-align: center;">
        <img alt="Julia" src="images/logos/julia-logo.svg" style="height: 2em; display: inline-block; margin: 1em;"/>
    </span>
    <span style="display: block; text-align: center;">
        Part 1, Example
    </span>
</h1>

<div style="text-align: center;">
    <div style="text-align: center; display: inline-block; vertical-align: middle;">
        Ludger Pähler <br/>
        <small>
        TUM, Chair of Aerodynamics and Fluid mechanics <br/>
            <a href="mailto:ludger.paehler@tum.de" target="_blank">ludger.paehler@tum.de</a>
        </small>
    </div>
    <div style="text-align: center; display: inline-block;">
        <img src="images/logos/tum-logo.svg" style="height: 8em; display: inline-block;  vertical-align: middle; margin: 1em;"/>
    </div>
</div>

<div style="text-align: center;">
    <p style="text-align: center; display: inline-block; vertical-align: middle;">
        Oliver Schulz<br>
        <small>
            Max Planck Institute for Physics <br/>
            <a href="mailto:oschulz@mpp.mpg.de" target="_blank">oschulz@mpp.mpg.de</a>
        </small>
    </p>
    <p style="text-align: center; display: inline-block; vertical-align: middle;">
        <img src="images/logos/mpg-logo.svg" style="height: 5em; display: inline-block; vertical-align: middle; margin: 1em;"/>
        <img src="images/logos/mpp-logo.svg" style="height: 5em; display: inline-block; vertical-align: middle; margin: 1em;"/>
    </p>
</div>

<p style="text-align: center;">
    Leibniz Supercomputing Centre - LRZ, Jan. 27th 2019
</p>

## Example: Local Gravity

In [ ]:
using VideoIO, Images, Colors, FileIO
using Statistics
using ArraysOfArrays
using Unitful, Measurements
using ArraysOfArrays, StaticArrays, StructArrays
using BenchmarkTools
using TypedTables
using Printf

In [ ]:
using Plots; gr(format = :png)
using Interact

In [ ]:
function read_video(filename::AbstractString)
    duration_in_s = VideoIO.get_duration(filename)
    input = VideoIO.openvideo(filename)
    try
        frames_per_ms = input.framerate
        nframes = floor(Int, duration_in_s * frames_per_ms / 1000) - 1
        @info "Reading \"$filename\" with $nframes frames"
        read_video(input, nframes)
    finally
        close(input)
    end
end


function read_video(input::VideoIO.VideoReader, nframes)
    frame = read(input)
    video = Array{eltype(frame)}(undef, size(frame)..., nframes)
    
    for i in axes(video, 3)
        read!(input, frame)
        view(video, :, :, i) .= frame
    end

    video
end

In [ ]:
video = read_video("ball_throw.mp4")
typeof(video); Base.format_bytes(sizeof(video))

In [ ]:
nestedview(video[1:5:end,1:5:end,1:20:end], Val(2))

In [ ]:
frames = nestedview(video, Val(2));

In [ ]:
plot(frames[50][85:115,300:335], ratio = 1)

In [ ]:
ball_color = HSV(mean(frames[50][85:115,300:335]))

In [ ]:
# correllate(a::RGB, b::RGB) = @fastmath sqrt(a.r * b.r + a.g * b.g + a.b * b.b)
correllate(a::RGB, b::HSL) = @fastmath (ca = HSL(a); sqrt((ca.h - b.h)^2 + (ca.s - b.s)^2))

In [ ]:
background = copy(frames[1]);

In [ ]:
framediff(f::Function, frame::AbstractArray, ref_frame::AbstractArray, ref_color::Color) =
    f.(correllate.(frame, ball_color) .- correllate.(ref_frame, ball_color))

framediff(frame::AbstractArray, ref_frame::AbstractArray, ref_color::Color) =
    framediff(identity, frame, ref_frame, ref_color)

In [ ]:
correllate(a::RGB, b::HSV) = @fastmath (ca = HSV(a); sqrt((Float32(ca.h - b.h)/360)^2 + Float32(ca.s - b.s)^2))
heatmap(framediff(frames[50], background, ball_color))

In [ ]:
heatmap(framediff(x -> x < -0.4, frames[50], background, ball_color))

In [ ]:
@time diffvideo = framediff(x -> x < -0.4, video, background, ball_color)
typeof(diffvideo), Base.format_bytes(sizeof(diffvideo))

In [ ]:
dframes = nestedview(diffvideo, Val(2));

In [ ]:
@time sum(((idx, w) -> SVector(Tuple(idx)) * w).(CartesianIndices(A), A)) / sum(A)

In [ ]:
function mean_pos(W::AbstractArray{T,N}) where {T,N}
    U = float(T)
    R = SVector{N,U}
    sum_pos::R = zero(R)
    sum_w::U = zero(U)
    @inbounds for idx in CartesianIndices(W)
        w = W[idx]
        sum_pos += SVector(Tuple(idx)) * w
        sum_w += w
    end
    sum_pos / sum_w
end

In [ ]:
@info mean_pos(A)
@benchmark mean_pos($A)

In [ ]:
struct Vec2D{T} <: FieldVector{2,T}
    x::T
    y::T
end

In [ ]:
PV = StructArray(Vec2D.(mean_pos.(dframes)))

In [ ]:
plot(PV.x, PV.y, yflip = true)

In [ ]:
framerate = 240
raw_data = Table(u = PV, t = (eachindex(PV) .- firstindex(PV)) / framerate)

In [ ]:
@manipulate for i in eachindex(dframes)
    white = eltype(background)(colorant"springgreen4")
    plot(((w,c) -> w ? white : c).(dframes[i], background)', ratio = 1)
    plot!(raw_data.u.x, raw_data.u.y, yflip = true, color = :blue, label = "trajectory")
    scatter!([raw_data.u.x[i]], [raw_data.u.y[i]], marker = (:xcross, :red), label = (@sprintf "%.2f s" raw_data.t[i]))
end

In [ ]:
sel_idxs = 27:244

In [ ]:
FileIO.save("background.png", background)

In [ ]:
raw_xy_shift = Vec2D(0, lastindex(axes(background,2)))

xy_cal_factor = 1.83 / 1559 * 1.72/1.82

xy_cal = SMatrix{2,2}(
    xy_cal_factor,             0,
                0, -xy_cal_factor
)

cal_data = Table(
    u = StructArray(Vec2D.(Ref(xy_cal) .* (raw_data.u .- Ref(raw_xy_shift)))),
    t = copy(collect(raw_data.t)),
)

# Fix missing frame:
view(cal_data.t, 169:lastindex(cal_data.t)) .+= 1 / framerate

sel_data = cal_data[sel_idxs]

scatter(
    sel_data.u.x, sel_data.u.y,
    marker = (:circle, 2, :black, stroke(0)),
    xlabel = "x [m]", ylabel = "y [m]"
)

In [ ]:
using CurveFit

In [ ]:
f = curve_fit(Poly, sel_data.t, sel_data.u.y, 2)

In [ ]:
plot(sel_data.t, f.(sel_data.t), label = "fit")
scatter!(
    sel_data.t, sel_data.u.y,
    marker = (:circle, 2, :black, stroke(0)),
    xlabel = "t [s]", ylabel = "y [m]",
    label = "data"
)

In [ ]:
g_curvefit = -2 * f.a[3] * u"m/s"

In [ ]:
using LinearAlgebra # For norm, etc.
using DifferentialEquations

In [ ]:
function motion_eq!(ddu, du, u, params, t)
    a = Vec2D(ddu)
    v = Vec2D(du)

    g, C_w = params
    
    rho_air = 1.209 # in kg/m^3, at 22°C and 1024 mbar
    d_ball = 60e-3 # m
    m_ball = 7.1e-3 # kg

    m = m_ball
    A = pi * (d_ball/2)^2 # cross section area

    f_drag = - rho_air * A * C_w * norm(v) * v / 2
    f_grav = m * Vec2D(zero(g), -g)

    a = (f_drag + f_grav) / m
   
    ddu[:] = a
end

#g_local = 9.81 # m/s^2
g_local = 9.83 # m/s^2

#C_w = 0.47 # unitless, typical value for sphere
C_w = 0.65

params = [g_local, C_w]

u0 = Array(sel_data.u[1])
du0 = Array((sel_data.u[2] - u0) / (sel_data.t[2] - sel_data.t[1]) + Vec2D(0.0, 0.38))

timespan = (first(sel_data.t), last(sel_data.t))

# r = solve(SecondOrderODEProblem{true}(motion_eq!, du0, u0, timespan, params), Tsit5(), saveat = sel_data.t)
r = solve(SecondOrderODEProblem{true}(motion_eq!, du0, u0, timespan, params), Tsit5())

sim_data = Table(
    # u = StructArray((u -> Vec2D(u.x[2])).(r.u)),
    # t = r.t,
    u = StructArray((x -> Vec2D(x.x[2])).(r.(sel_data.t))),
    t = sel_data.t,
)

plot(sim_data.t, sim_data.u.y, #=marker = (:circle, 2, stroke(0)), label = "sim"=#)
scatter!(
    sel_data.t, sel_data.u.y,
    marker = (:circle, 2, :black, stroke(0)),
    xlabel = "t [s]", ylabel = "y [m]",
    label = "data"
)

In [ ]:
@benchmark solve(SecondOrderODEProblem{true}(motion_eq!, du0, u0, timespan, params), Tsit5())

## With Units

In [ ]:
function motion_eq!(ddu, du, u, params, t)
    a = Vec2D(ddu)
    v = Vec2D(du)

    g, C_w = params
    
    rho_air = 1.209u"kg/m^3"# at 22°C and 1024 mbar
    d_ball = 60e-3u"m"
    m_ball = 7.1e-3u"kg"

    m = m_ball
    A = π * (d_ball/2)^2 # cross section area

    f_drag = - rho_air * A * C_w * norm(v) * v / 2
    f_grav = m * Vec2D(zero(g), -g)

    a = (f_drag + f_grav) / m
   
    ddu[:] = a
end

#g_local = 9.81u"m/s^2"
g_local = 9.83u"m/s^2"

#C_w = 0.47 # unitless, typical value for sphere
C_w = 0.65

params = [g_local, C_w]

timespan = (first(sel_data.t)u"s", last(sel_data.t)u"s")

u0 = Array(sel_data.u[1])u"m"
du0 = Array((sel_data.u[2]u"m" - u0) / (sel_data.t[2] - sel_data.t[1])u"s" + Vec2D(0.0, 0.38)u"m/s")

#r = solve(SecondOrderODEProblem{true}(motion_eq!, du0, u0, timespan, params), Midpoint(), saveat = sel_data.t * u"s")
r = solve(SecondOrderODEProblem{true}(motion_eq!, du0, u0, timespan, params), Midpoint())

sim_data = Table(
    u = StructArray((u -> Vec2D(u.x[2])).(r.u)),
    t = r.t,
    #u = StructArray((x -> Vec2D(x.x[2])).(r.(sel_data.t * u"s"))),
    #t = sel_data.t * u"s",
)

using UnitfulRecipes
plot(sim_data.t, sim_data.u.y, xlabel = "t", ylabel = "y")

In [ ]:
@benchmark solve(SecondOrderODEProblem{true}(motion_eq!, du0, u0, timespan, params), Midpoint())